In [3]:
import zarr
import numpy as np
import pandas as pd

In [2]:
enformer = pd.read_table('/data1/deepLN/Enformer/Korean_CSM._enformer_5313_features.txt')

In [3]:
dnv = pd.read_table('/home/yjkim98/motif_analysis/Korean_CSM.WGS.autosomal_DNV.178samples.sorted.20240426.vcf')

In [4]:
dnv = dnv.drop(columns=['#CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'FILTER'])

In [5]:
dnv.rename(columns={'ID': 'variant'}, inplace=True)

In [6]:
dnv['samples'] = dnv['INFO'].str.extract(r'SAMPLE=([^;]+)')

In [7]:
dnv = dnv.drop(columns=['INFO'])

In [8]:
dnv

,variant,samples
0,chr1:1196415:T:C,IBS-ASD-17104-blood-wgs-ILLUMINA
1,chr1:1267957:C:T,IBS-ASD-17143-blood-wgs-ILLUMINA
2,chr1:1339884:A:ACAGCCGCATGTCCCCCCGCAGCCCCCCCAG...,IBS-ASD-18513-blood-wgs-ILLUMINA
3,chr1:1778916:G:T,IBS-ASD-20023-blood-wgs-ILLUMINA
4,chr1:2178615:C:T,IBS-ASD-16603-blood-wgs-ILLUMINA
...,...,...
12286,chr22:49965151:C:T,IBS-ASD-19704-blood-wgs-ILLUMINA
12287,chr22:50055947:G:A,IBS-ASD-16603-blood-wgs-ILLUMINA
12288,chr22:50482853:C:T,IBS-ASD-18415-blood-wgs-ILLUMINA
12289,chr22:50484353:G:A,IBS-ASD-19903-blood-wgs-ILLUMINA


In [9]:
# 'id' 열을 'variant'로 이름 변경
enformer.rename(columns={'id': 'variant'}, inplace=True)

In [2]:
# 'chrom', 'pos', 'ref', 'alt' 열 삭제
enformer.drop(columns=['chrom', 'pos', 'ref', 'alt'], inplace=True)

NameError: name 'enformer' is not defined

In [11]:
# DataFrame을 고유(unique)하게 만듦 (중복 행 제거)
enformer = enformer.drop_duplicates()

In [12]:
# 'variant' 열을 기준으로 dnv와 enformer DataFrame을 병합
merged_df = pd.merge(dnv, enformer, on='variant', how='inner')

In [13]:
print('Load Enformer', flush=True)

enformer_path = '/data1/deepLN/Enformer/enformer_result_kor_sfari_mssng.zarr'

root = zarr.open(enformer_path, mode='r')
mat = root['data']
column_names1 = root['metadata'].attrs['columns']
var_ids = root['metadata'].attrs['variant']
samples = root['metadata'].attrs['samples']

print(mat.shape, flush=True)

# Create a DataFrame
enformer_d = pd.DataFrame(
    data=mat,
    columns=column_names1
)

enformer_d['variant'] = var_ids
enformer_d['samples'] = samples

Load Enformer
(954915, 5313)


In [14]:
# 두 DataFrame의 열 순서를 맞춘 후 행 결합
merged_df = merged_df[enformer_d.columns]  # enformer의 열 순서에 맞게 merged_df 재정렬
combined_df = pd.concat([enformer_d, merged_df], ignore_index=True)  # 행 결합

In [16]:
output_path = '/data1/deepLN/Enformer/enformer_result_kor_909_sfari_mssng.zarr'
root = zarr.open(output_path, mode='w')

# Create metadata group and add attributes
root.create_group('metadata')
root['metadata'].attrs['variant'] = combined_df['variant'].tolist()
sample_ids = combined_df['samples'].tolist()
root['metadata'].attrs['samples'] = sample_ids

# Drop non-numeric columns and get column names
combined_dataframe = combined_df.drop(['variant', 'samples'], axis=1)
column_names = combined_dataframe.columns.tolist()
root['metadata'].attrs['columns'] = column_names

# Ensure data is in float format and create dataset
root.create_dataset('data', data=combined_dataframe.values, chunks=(1000, 1000), dtype='float')
print(f'Data saved to {output_path}', flush=True)

Data saved to /data1/deepLN/Enformer/enformer_result_kor_909_sfari_mssng.zarr
